In [2]:
import pandas as pd
import os
import glob
import concurrent.futures

directory = 'C:\\Users\\Trieu Pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
directory2 = 'C:\\Users\\trieu.pham\\Desktop\\230105 Payment Summary - 2023-01-06T085450.177'
cate_dir = 'C:\\Users\\Trieu Pham\\Desktop\\DataCate.xlsx'
cate_dir2 = 'C:\\Users\\trieu.pham\\Desktop\\DataCate.xlsx'

# list of Excel files to read data from
if os.path.exists(os.path.join(directory, '230101 Payment Summary - 2023-01-03T093340.600.xlsx')):
    excel_files = glob.glob(os.path.join(directory, '*.xlsx'))
else:
    excel_files = glob.glob(os.path.join(directory2, '*.xlsx'))
try:
    # use cols 0, Barcode; 2, Category Name
    # df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir, sheet_name='Sheet1', usecols=[0,3])
    pass
except FileNotFoundError:
    # df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,2])
    # use cols 0, Barcode; 3, SubCategory Name
    df2 = pd.read_excel(cate_dir2, sheet_name='Sheet1', usecols=[0,3])
    pass
# create an empty list to store the dataframes
# dfs = [] sho

outputfile =  'dataT1.csv'
# def excel2csv
def excel2csv(inputfile, outputfile):
    
    # Check if exist file_path
    if not os.path.exists(outputfile):
        with open(outputfile, 'w') as f:
            pass

    # Read the first line of the file
    with open(outputfile, 'r') as f:
        first_line = f.readline().strip()
    # Check if the first line contains the expected column names
    has_headers = 'InvoiceID' in first_line
    df = pd.read_excel(inputfile, sheet_name='Sheet2', skiprows=2, usecols=[4,7,8,14])
    # df2 Category
    
    # merge with Category, on "Barcode"
    df = df.merge(df2, on='Barcode', how='left')
    
    # Check if the first line contains the expected column names
    df.to_csv(outputfile, mode='a', header= not has_headers, index=False)


# loop through the list of Excel files
with concurrent.futures.ThreadPoolExecutor() as executor:
    [executor.submit(excel2csv, file, outputfile) for file in excel_files]
    # for file in excel_files:
    #     executor.submit(excel2csv,file)

In [38]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Sample data
dtypes = {'InvoiceID': str, 'Barcode': str, 'Sum of Qty': str, 'Total Exclude VAT': str, 'SubCategory Name': str}
df = pd.read_csv('~/Desktop/Data/dataT1.csv', dtype=dtypes)

df['Sum of Qty'] = pd.to_numeric(df['Sum of Qty'], errors='coerce').fillna(0).astype(int)
df['Total Exclude VAT'] = pd.to_numeric(df['Total Exclude VAT'], errors='coerce').fillna(0).astype(float)
df['SubCategory Name'] = df['SubCategory Name'].astype(str)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1844699 entries, 0 to 1844698
Data columns (total 5 columns):
 #   Column             Dtype  
---  ------             -----  
 0   InvoiceID          object 
 1   Barcode            object 
 2   Sum of Qty         int64  
 3   Total Exclude VAT  float64
 4   SubCategory Name   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 70.4+ MB


In [39]:
# bill0 = df1.groupby('InvoiceID').filter(lambda x: x['Sum of Qty'].sum() == 0)

bill0 = df[df['Sum of Qty'] == 0]

bill0

,InvoiceID,Barcode,Sum of Qty,Total Exclude VAT,SubCategory Name
51,VN0001010201230023,260501000011,0,0.0,Hot pot Thai
119,VN0001010201230057,8936127796217,0,0.0,Sugar Tea
120,VN0001010201230057,8936127796019,0,0.0,Sugar Tea
262,VN0001010201230116,2010809000482,0,0.0,Cold Local Drinks
378,VN0001010201230168,8935049510864,0,0.0,Purified water
...,...,...,...,...,...
1843975,VN0006010101230039,8934692090020,0,0.0,Import Beer Can
1844291,VN0006020101230004,8809640030611,0,0.0,Distilled Spirits
1844292,VN0006020101230004,8801048178308,0,0.0,Distilled Spirits
1844350,VN0006020101230025,8934588013133,0,0.0,Sugar CSD


In [64]:
# filter out rows where 'Sum of Qty' or 'Total Exclude VAT' are zero
df_filtered = df[(df['Sum of Qty'] > 0) & (df['Total Exclude VAT'] > 0)]

new_df = df_filtered.dropna().groupby('InvoiceID').agg({'Barcode': lambda x: ', '.join(x.astype(str)), 
                                      'Sum of Qty': 'sum', 
                                      'Total Exclude VAT': 'sum', 
                                      'SubCategory Name': lambda x: ', '.join(x.astype(str))})
new_df


,Barcode,Sum of Qty,Total Exclude VAT,SubCategory Name
InvoiceID,,,,
VN0001010101230001,8935049500544,1,6481.0,Mineral water
VN0001010101230002,8938512632025,2,11112.0,Purified water
VN0001010101230003,"8936011773416, 8936079121761, 8850453017528, 8...",5,106482.0,"Other Ice Cream, Potato Snacks, Stick, UHT Mil..."
VN0001010101230004,"8801100128845, 2010101000005, 2010805000295, 8...",6,174478.0,"Distilled Spirits, Cup Of Ice, Add-on item Sub..."
VN0001010101230005,"8934588843051, 8934588183133",2,24074.0,"Sports Drinks Sub, Sports Drinks Sub"
...,...,...,...,...
VN0236021201230081,8934588223556,1,10909.0,Sugar Tea
VN0236021201230082,8934564300127,1,10000.0,Sandwich cookies
VN0236021201230083,6915878221577,2,85455.0,Import Beer Bottle


In [65]:
new_df.to_csv('dataCleanM1.csv')